## Installers here ##

In [1]:
#! pip install transformers
#! pip install langchain-chroma
 #! pip install langchain-groq

In [2]:
#! pip install -U langchain-huggingface sentence-transformers

## Loading the documents into the RAG ##

In [3]:
# Load the documents

from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader

loader = DirectoryLoader('data/alzheimers', glob="*.txt", show_progress=True, loader_cls=TextLoader)

kb_docs = loader.load()

100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 378.74it/s]


In [4]:
# Chunk the loaded documents

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=50)

chunks = text_splitter.split_documents(kb_docs)

In [5]:
# hugging face embedding model is used as I am not having openAI key access and usage restrictions.
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",  # Free, local
    # model_kwargs={"device": "cuda"}  # GPU acceleration
)

In [6]:
# Step 1: Initialize the Chroma DB Connection
# Initialize a ChromaDB Connection
from langchain_chroma import Chroma

# Initialize the database connection
# If database exist, it will connect with the collection_name and persist_directory
# Otherwise a new collection will be created
db = Chroma(collection_name="vector_database", 
            embedding_function=embedding_model, 
            persist_directory="./chroma_db_")

In [7]:
# Insert chunks in the vector database

db.add_documents(documents=chunks)

['9b0e8009-9908-4e53-bb7c-aa7801d44fc7',
 'b0567ecd-3f01-4caa-95a3-3786660a7109',
 '3c3ecd06-5217-4e99-b7df-129796c4aae7',
 'd178f307-9836-4958-a5b7-7abd5f619d7b',
 '96ff63b9-6d54-43c4-aad1-069b1af00d3f',
 'a1dd825d-0b79-4669-8252-fbef7886bfca',
 '99e970e8-f0f6-4070-b81d-5554bfeb082e',
 '2f293577-e84f-4e32-9d07-da8056345846',
 'a9f9b67e-9601-44ba-abe8-887f96ded24e',
 '075430ca-9745-4c80-9ab7-34816b99c5f4',
 '617506a2-5239-45a8-8367-0422910494d1',
 'b5bc71dd-ce03-4f3a-a835-18c2c0ccdef0',
 '2784510c-3a85-41c8-8bd4-2c18dccb0e4f',
 '0423b71d-6a3e-4e9f-97c4-aa6b47458379',
 '064beaf9-b618-48a2-8f0c-9aa29fe3b25b',
 'f4c106fd-ffa5-4563-9a73-964b56e0a61b',
 'df6c6fd0-e204-49e0-bf14-61a152c8628a',
 'b38eb7eb-9103-40e1-97d4-a550c86775da',
 '4e88ff08-5eb5-4b06-a05e-f747e92b11cf',
 '9987eef4-d6ca-48fc-a4e6-39f0752938d6',
 '7067cb12-5feb-473d-b747-ac24bba57579',
 '448fe996-b407-4c8a-9ff4-da0896bfd885',
 '6e027a3c-5df6-450f-b29e-e381c8ab81bd',
 '24ea610b-591d-4b5f-834b-27b7577b9c23',
 'a5c3fa35-40b2-

In [8]:
# Step 2: Create a Retriever Object 

retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [9]:
retriever.invoke("explain design patterns ")

[Document(id='84d66514-34e6-4dba-9569-c5510aa91d66', metadata={'source': 'data\\alzheimers\\alzheimers_3.txt'}, page_content='Techniques'),
 Document(id='ce62ba6f-7afe-4660-a784-93f6f9a852c8', metadata={'source': 'data\\alzheimers\\alzheimers_3.txt'}, page_content='Techniques'),
 Document(id='efb42a31-115e-466a-b67e-d07af6d6a1b0', metadata={'source': 'data\\alzheimers\\alzheimers_2.txt'}, page_content='Disease mechanism'),
 Document(id='169de9a6-9b82-4261-a388-f63a57005cb4', metadata={'source': 'data\\alzheimers\\alzheimers_2.txt'}, page_content='Disease mechanism')]

In [10]:
# Step 3: Initialize a Chat Prompt Template

from langchain_core.prompts import ChatPromptTemplate

PROMPT_TEMPLATE = """
Answer the question based only on the following context:
{context}
Answer the question based on the above context: {question}.

show the context that was passed.
Provide a detailed answer.
Don’t justify your answers.
Don’t give information not mentioned in the CONTEXT INFORMATION.
Do not say "according to the context" or "mentioned in the context" or similar.
"""

prompt_template = ChatPromptTemplate(
    messages=[
        PROMPT_TEMPLATE
    ]
)

In [11]:
! pip install -U langchain-huggingface huggingface_hub

  Using cached huggingface_hub-1.4.1-py3-none-any.whl.metadata (13 kB)



[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
# Step 4: Initialize a Generator (i.e. Chat Model)
from langchain_huggingface import HuggingFaceEndpoint
from langchain_huggingface import ChatHuggingFace  # Optional chat wrapper

f = open("keys/.huggingface_token.txt")
HUGGINGFACE_TOKEN = f.read()

llm = HuggingFaceEndpoint(
    temperature=0.7,
    model="Qwen/Qwen2.5-7B-Instruct",
    task="text-generation",
    huggingfacehub_api_token=HUGGINGFACE_TOKEN  # Free token from huggingface.co
)

chat_model = ChatHuggingFace(llm=llm)  # Chat interface like ChatOpenAI



In [13]:
chat_model.invoke("hello")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 30, 'total_tokens': 40}, 'model_name': 'Qwen/Qwen2.5-7B-Instruct', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c8548-49b6-7333-be12-81b3fa0b632e-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 30, 'output_tokens': 10, 'total_tokens': 40})

In [14]:

from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [15]:
generator_chain = prompt_template | chat_model 

In [16]:
# Helper function to join the retrieved chunks

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [17]:
# Step 6: Define a RAG Chain

from langchain_core.runnables import RunnablePassthrough

rag_chain = {
    "context": retriever | format_docs, 
    "question": RunnablePassthrough()
} | generator_chain

In [18]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

# Your original inputs
inputs = {
    "context": retriever | format_docs,
    "question": RunnablePassthrough()
}

# The original chain to generate the answer from the inputs
rag_chain = inputs | generator_chain

# New chain that returns both the context and the final answer
rag_chain_with_context = RunnableParallel(
    answer=rag_chain,                # final output
    context=inputs["context"],       # resolved context
    question=inputs["question"],     # optional, keep if you want to see it too
)

result = rag_chain_with_context.invoke("What won 1983 cricket worldcup?")
print(result.keys())
print("Context sent to the LLM:\n", result["context"])
print("\nAnswer:\n", result["answer"])
print("\nQuestion:\n",result["question"])

dict_keys(['answer', 'context', 'question'])
Context sent to the LLM:
 Techniques

Techniques

Disease mechanism

Disease mechanism

Answer:
 content='The context provided does not contain any information about the 1983 cricket worldcup.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 122, 'total_tokens': 142}, 'model_name': 'Qwen/Qwen2.5-7B-Instruct', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019c8548-565d-7b91-9bee-ce44955a1632-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 122, 'output_tokens': 20, 'total_tokens': 142}

Question:
 What won 1983 cricket worldcup?


In [19]:
# Invoke the Chain

query = 'explain about Fifa WorldCup?'

result = rag_chain.invoke(query)

In [20]:
print(result)

content='The context provided does not contain any information about the FIFA World Cup. The context is about disease mechanisms and techniques related to them.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 119, 'total_tokens': 146}, 'model_name': 'Qwen/Qwen2.5-7B-Instruct', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019c8548-6054-7c51-abaa-5987ef9a37a6-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 119, 'output_tokens': 27, 'total_tokens': 146}


In [21]:
query = "how to play football"
rag_chain.invoke(query)

AIMessage(content="The context provided does not contain any information about how to play football. The given text discusses intellectual activities and social interactions in relation to Alzheimer's disease risk but does not mention football or provide instructions on how to play it.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 177, 'total_tokens': 222}, 'model_name': 'Qwen/Qwen2.5-7B-Instruct', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c8548-643f-7542-ae38-b0ee9d21533b-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 177, 'output_tokens': 45, 'total_tokens': 222})